In [ ]:
import pandas as pd

file_path = r"C:\Users\rouat\Desktop\MMcows\Datasets\sensor_data\sensor_data\sub_data\health_records_cleaned\health_records_cleaned_using_llm\C01.csv"

df = pd.read_csv(file_path)

# Afficher les 5 premières lignes
print(df.head())


               Date       Event   Specific event  \
0  2023-08-31 23:00        Sale             Cull   
1  2023-08-21 02:00        Move             Move   
2  2023-08-21 00:00   Pregnancy  Pregnancy Check   
3  2023-08-16 00:00        Hoof             Trim   
4  2023-08-11 00:00  Medication          Off Exp   

                       Description  
0  Sold - Not reported Equity 6255  
1                            Pen 6  
2     Pregnant - Female - 116 days  
3                             Trim  
4                          Off Exp  


In [83]:
print("Nombre de lignes et colonnes :", df.shape)


Nombre de lignes et colonnes : (182, 4)


In [ ]:


df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

date_min = df['Date'].min()
date_max = df['Date'].max()

print(f"Période minimale : {date_min}")
print(f"Période maximale : {date_max}")

# Valeurs uniques pour 'Event'
unique_events = df['Event'].dropna().unique()
print(f"\nValeurs uniques pour 'Event' ({len(unique_events)}):")
print(unique_events)

# Valeurs uniques pour 'Specific event'
unique_specific_events = df['Specific event'].dropna().unique()
print(f"\nValeurs uniques pour 'Specific event' ({len(unique_specific_events)}):")
print(unique_specific_events)

# Valeurs uniques pour 'Description'
unique_descriptions = df['Description'].dropna().unique()
print(f"\nValeurs uniques pour 'Description' ({len(unique_descriptions)}):")
print(unique_descriptions)


Période minimale : 2016-12-15 00:00:00
Période maximale : 2023-08-31 23:00:00

Valeurs uniques pour 'Event' (20):
['Sale' 'Move' 'Pregnancy' 'Hoof' 'Medication' 'Breeding' 'Protocol' 'ID'
 'Health' 'VWP' 'Milk' 'Calving' 'Watch' 'Dry' 'Note' 'Vaccine' 'Weight'
 'Treatment' 'Injury' 'Birth']

Valeurs uniques pour 'Specific event' (26):
['Cull' 'Move' 'Pregnancy Check' 'Trim' 'Off Exp' 'On Exp' 'Breeding'
 'Protocol' 'ID Change' 'Lameness' 'VWP Expiration' 'Withhold Milk'
 'Milk Culture' 'Mastitis' 'Office Call' 'Off Feed' 'Freshen' 'Move To'
 'Watch List' 'Dry Off' 'Note' 'Vaccine' 'Weight' 'Treatment' 'Injury'
 'Birth']

Valeurs uniques pour 'Description' (126):
['Sold - Not reported Equity 6255' 'Pen 6' 'Pregnant - Female - 116 days'
 'Trim' 'Off Exp' 'On Exp - 683CC' 'Pen 4A' 'Pregnant - Female - Heifer'
 'Pregnant - Unknown - 46 days' 'Off Exp - HMW668'
 'Pregnant - Unknown - 32 days' '829XB0921 - 829XB921 by Tech 19'
 'Double Ovsynch - Completed' 'Open - 32 days' 'On Exp - HMW668'


In [ ]:

# Afficher, pour chaque Event, la liste des Specific event correspondants
grouped = df.groupby('Event')['Specific event'].unique()

for event, specific_events in grouped.items():
    print(f"\nEvent: {event}")
    print(f"Specific events: {specific_events}")



Event: Birth
Specific events: ['Birth']

Event: Breeding
Specific events: ['Breeding']

Event: Calving
Specific events: ['Freshen']

Event: Dry
Specific events: ['Dry Off']

Event: Health
Specific events: ['Lameness' 'Milk Culture' 'Mastitis' 'Office Call' 'Off Feed']

Event: Hoof
Specific events: ['Trim']

Event: ID
Specific events: ['ID Change']

Event: Injury
Specific events: ['Injury']

Event: Medication
Specific events: ['Off Exp' 'On Exp']

Event: Milk
Specific events: ['Withhold Milk']

Event: Move
Specific events: ['Move' 'Move To']

Event: Note
Specific events: ['Note']

Event: Pregnancy
Specific events: ['Pregnancy Check']

Event: Protocol
Specific events: ['Protocol']

Event: Sale
Specific events: ['Cull']

Event: Treatment
Specific events: ['Treatment']

Event: VWP
Specific events: ['VWP Expiration']

Event: Vaccine
Specific events: ['Vaccine']

Event: Watch
Specific events: ['Watch List']

Event: Weight
Specific events: ['Weight']


In [ ]:
# ----- AVANT nettoyage -----
print("=== AVANT NETTOYAGE ===")
# 'Event'
unique_event_before = df['Event'].unique()
print(f"\nNombre de valeurs uniques pour 'Event' avant nettoyage : {len(unique_event_before)}")
print("Valeurs uniques pour 'Event' avant nettoyage :")
print(unique_event_before)

# 'Specific event'
unique_specific_before = df['Specific event'].unique()
print(f"\nNombre de valeurs uniques pour 'Specific event' avant nettoyage : {len(unique_specific_before)}")
print("Valeurs uniques pour 'Specific event' avant nettoyage :")
print(unique_specific_before)

# ----- Dictionnaires de normalisation -----
event_mapping = {
    'Meat': 'Sale', 'Meat Hold': 'Sale', 'Medication': 'Health', 'Treatment': 'Health',
    'Medical diagnosis': 'Health', 'Surgery': 'Health', 'Vaccination': 'Vaccine', 'Vaccine': 'Vaccine',
    'Repro': 'Reproduction', 'Repro Check': 'Reproduction', 'Pregnancy': 'Reproduction',
    'Pregnancy Check': 'Reproduction', 'Heat': 'Reproduction', 'DNB': 'Reproduction',
    'Dry Off': 'Dry', 'Dry': 'Dry', 'Hoof Maintenance': 'Hoof', 'Hoof': 'Hoof',
    'Enroll in protocol': 'Protocol', 'Protocol': 'Protocol', 'Watch List': 'Watch', 'Watch': 'Watch',
    'Change EID': 'ID', 'ID': 'ID', 'Birth': 'Calving', 'Freshen': 'Calving', 'Calving': 'Calving',
    'VWP expiration': 'VWP', 'VWP Expiration': 'VWP', 'VWP': 'VWP',
    'History': 'Note', 'Prev Lactation': 'Note', 'Notes': 'Note', 'Note': 'Note', 'Other': 'Note',
    'Sale': 'Sale', 'Move': 'Move', 'Move To': 'Move', 'Weight': 'Weight', 'Injury': 'Injury',
    'Withhold Milk': 'Milk'
}

specific_event_mapping = {
    'Move To': 'Move', 'Birth': 'Freshen', 'Repro Check': 'Pregnancy Check',
    'VWP expiration': 'VWP Expiration', 'VWP Expiration': 'VWP Expiration',
    'Vaccination': 'Vaccine', 'Notes': 'Note', 'Prev Lactation': 'Note',
    'Cull': 'Sale', 'Meat Hold': 'Sale', 'Change EID': 'ID Change'
}

# ----- Nettoyage -----
df_normalized = df.copy()
df_normalized['Event'] = df_normalized['Event'].replace(event_mapping)
df_normalized['Specific event'] = df_normalized['Specific event'].replace(specific_event_mapping)

# ----- APRES nettoyage -----
print("\n=== APRES NETTOYAGE ===")
# 'Event'
unique_event_after = df_normalized['Event'].unique()
print(f"\nNombre de valeurs uniques pour 'Event' après nettoyage : {len(unique_event_after)}")
print("Valeurs uniques pour 'Event' après nettoyage :")
print(unique_event_after)

unique_specific_after = df_normalized['Specific event'].unique()
print(f"\nNombre de valeurs uniques pour 'Specific event' après nettoyage : {len(unique_specific_after)}")
print("Valeurs uniques pour 'Specific event' après nettoyage :")
print(unique_specific_after)



=== AVANT NETTOYAGE ===

Nombre de valeurs uniques pour 'Event' avant nettoyage : 20
Valeurs uniques pour 'Event' avant nettoyage :
['Sale' 'Move' 'Pregnancy' 'Hoof' 'Medication' 'Breeding' 'Protocol' 'ID'
 'Health' 'VWP' 'Milk' 'Calving' 'Watch' 'Dry' 'Note' 'Vaccine' 'Weight'
 'Treatment' 'Injury' 'Birth']

Nombre de valeurs uniques pour 'Specific event' avant nettoyage : 26
Valeurs uniques pour 'Specific event' avant nettoyage :
['Cull' 'Move' 'Pregnancy Check' 'Trim' 'Off Exp' 'On Exp' 'Breeding'
 'Protocol' 'ID Change' 'Lameness' 'VWP Expiration' 'Withhold Milk'
 'Milk Culture' 'Mastitis' 'Office Call' 'Off Feed' 'Freshen' 'Move To'
 'Watch List' 'Dry Off' 'Note' 'Vaccine' 'Weight' 'Treatment' 'Injury'
 'Birth']

=== APRES NETTOYAGE ===

Nombre de valeurs uniques pour 'Event' après nettoyage : 17
Valeurs uniques pour 'Event' après nettoyage :
['Sale' 'Move' 'Reproduction' 'Hoof' 'Health' 'Breeding' 'Protocol' 'ID'
 'VWP' 'Milk' 'Calving' 'Watch' 'Dry' 'Note' 'Vaccine' 'Weight' 'In

In [ ]:

# Afficher, pour chaque Event, la liste des Specific event correspondants
grouped = df_normalized.groupby('Event')['Specific event'].unique()

for event, specific_events in grouped.items():
    print(f"\nEvent: {event}")
    print(f"Specific events: {specific_events}")



Event: Breeding
Specific events: ['Breeding']

Event: Calving
Specific events: ['Freshen']

Event: Dry
Specific events: ['Dry Off']

Event: Health
Specific events: ['Off Exp' 'On Exp' 'Lameness' 'Milk Culture' 'Mastitis' 'Office Call'
 'Off Feed' 'Treatment']

Event: Hoof
Specific events: ['Trim']

Event: ID
Specific events: ['ID Change']

Event: Injury
Specific events: ['Injury']

Event: Milk
Specific events: ['Withhold Milk']

Event: Move
Specific events: ['Move']

Event: Note
Specific events: ['Note']

Event: Protocol
Specific events: ['Protocol']

Event: Reproduction
Specific events: ['Pregnancy Check']

Event: Sale
Specific events: ['Sale']

Event: VWP
Specific events: ['VWP Expiration']

Event: Vaccine
Specific events: ['Vaccine']

Event: Watch
Specific events: ['Watch List']

Event: Weight
Specific events: ['Weight']


In [ ]:
def attribuer_categorie(description):
    description = str(description).lower() 
    if any(mot in description for mot in ['pen', 'corrid', 'stall']):
        return 'Location'
    elif any(mot in description for mot in ['aspirin', 'banamine', 'treatment', 'ketosis']):
        return 'Treatment'
    elif any(mot in description for mot in ['trim', 'hoof', 'foot']):
        return 'Hoof Care'
    elif any(mot in description for mot in ['pregnant', 'heat', 'fresh cow']):
        return 'Reproductive Status'
    elif any(mot in description for mot in ['protocol', 'breeding', 'ovsynch']):
        return 'Protocol'
    elif any(mot in description for mot in ['fecal', 'urine', 'test', 'culture']):
        return 'Lab Tests'
    elif any(mot in description for mot in ['milk', 'withhold', 'hold']):
        return 'Milk Management'
    elif any(mot in description for mot in ['note', 'comment', 'off feed']):
        return 'Comment'
    else:
        return 'Other'

df_normalized['Description_category'] = df_normalized['Description'].apply(attribuer_categorie)

#  normaliser la colonne Description 
df_normalized['Description'] = df_normalized['Description'].replace(
    to_replace=r'Pregnant.*', value='Pregnant', regex=True)

df_normalized['Description'] = df_normalized['Description'].replace(
    ['Banamine', 'Gentamicin', 'Naxcel', 'Draxxin'], 'Antibiotic')

df_normalized['Description'] = df_normalized['Description'].replace(
    to_replace=r'Aspirin.*', value='Aspirin', regex=True)

df_normalized['Description'] = df_normalized['Description'].replace(
    to_replace=r'Fresh Cow.*', value='Fresh Cow', regex=True)

df_normalized['Description'] = df_normalized['Description'].replace(
    to_replace=r'Open.*', value='Open', regex=True)

df_normalized['Description'] = df_normalized['Description'].replace(
    to_replace=r'New:.*Old:.*', value='EID Change', regex=True)


In [97]:
print ( df_normalized.head())

                 Date         Event   Specific event  \
0 2023-08-31 23:00:00          Sale             Sale   
1 2023-08-21 02:00:00          Move             Move   
2 2023-08-21 00:00:00  Reproduction  Pregnancy Check   
3 2023-08-16 00:00:00          Hoof             Trim   
4 2023-08-11 00:00:00        Health          Off Exp   

                       Description Description_category  
0  Sold - Not reported Equity 6255                Other  
1                            Pen 6             Location  
2                         Pregnant  Reproductive Status  
3                             Trim            Hoof Care  
4                          Off Exp                Other  


In [101]:
output_path = r"C:\Users\rouat\Desktop\MMcows\Datasets\sensor_data\sensor_data\sub_data\health_records_cleaned\health_records_cleaned_manually\C01.csv"
df_normalized.to_csv(output_path, index=False)
